In [1]:
import numpy as np
import pandas as pd
from Score import rouge_1,rouge_2,rouge_l,bleu_4,score
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizer, BertModel, BertConfig

In [34]:
# 加载本地Bert模型
# Load the BERT model and tokenizer
config = BertConfig.from_json_file("../Bert_model/Bert/config.json")
tokenizer = BertTokenizer.from_pretrained('../Bert_model/Bert/')
bert = BertModel.from_pretrained("../Bert_model/Bert/", config=config)

In [35]:
# 获取词向量
def get_word_embeddings(text, model, token):
    text = "CLS" + text + "SEP"
    encoded_inputs = token(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # encoded_inputs = {k: v.to(device) for k, v in encoded_inputs.items()}

    # Pretrain the text
    with torch.no_grad():
        outputs = model(**encoded_inputs)

    # Get the CLS token embeddings
    # 将cls标记的向量作为整个句子向量
    word_embedding = outputs[0][:, 0, :]

    return word_embedding

In [36]:
def semantic_similarity(answer_1, answer_2, model=bert, token=tokenizer):
    a = 0
    try:
        s1 = get_word_embeddings(answer_1, model, token)
        s2 = get_word_embeddings(answer_2, model, token)
        # 计算余弦相似度
        similarity = cosine_similarity(s1.cpu(), s2.cpu())
        a = round(np.mean(np.mean(similarity, axis=1)), 4)
        # print(f"第{i+1}段文本的语义相似度为: {a:.4f}")
    except Exception as e3:
        print(f"相似度计算的错误为{e3}")

    return a

读取数据

In [81]:
data = pd.read_csv('data_1000.csv').iloc[:,:4]

获取ChatGPT回答

In [69]:
from openai import OpenAI
# from time import sleep
client = OpenAI(
    base_url="XXXXXX",
    api_key='XXXXXX',
)

In [71]:
def generate_response(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user","content": user_input,}
        ],
        model="gpt-3.5-turbo",
        max_tokens=512,
        )
    message=chat_completion.choices[0].message.content
    
    # message=chat_completion.choices[0]
    return message.strip().replace("\n", "")

In [78]:
#创建新数据集存储生成的多个答案
data_GPT=pd.DataFrame(columns=['问题','医生回答','GPTanswer_1','c1','GPTanswer_2','c2','GPTanswer_3','c3','GPTanswer_4','c4','GPTanswer_5','c5'])

In [79]:
answer_name=['GPTanswer_1','GPTanswer_2','GPTanswer_3','GPTanswer_4','GPTanswer_5']
c_name=['c1','c2','c3','c4','c5']

In [2]:
# for index in range(961,len(data)):
# #保存回答和问题在新的数据集
#     data_GPT.at[index,'问题']=data['问题'][index]
#     data_GPT.at[index,'医生回答']=data['医生回答'][index]
#     
#     question= data["问题"][index]
#     # instruction="针对给出的问题，给出更可靠的答案"
#     
#     answer1=generate_response(question)
#     # c1=round(score([question],[answer1]).iloc[0,0],4)
#     
#     data_GPT.at[index,answer_name[0]]=answer1
#     # data_GPT.at[index,c_name[0]]=c1
#     
#     # in_put=f"问题：{question}\n回答1：{answer1}\n得分：{c1}\n"
#     
#     for i in range(1,5):
#         try:
#             answer=generate_response(question)
#             # c=round(score([question],[answer]).iloc[0,0],4)
#             
#             data_GPT.at[index,answer_name[i]]=answer
#             # data_GPT.at[index,c_name[i]]=c
#             # in_put=in_put+f"回答{i+1}:{answer}\n得分：{c}\n"
#                 
#             print(f"第{index}次问答")
#         except Exception as e:
#             print(f"Error occurred for question at index {index}: {e}")
#             answer = None
#             # c=None
# data_GPT.to_csv('chatgpt问答_5_answer_2.csv')

In [2]:
# best_answers = pd.read_csv('best_answers.csv')

In [3]:
# rouge_1_score=rouge_1(best_answers['L5'],data_GPT['医生回答'])
# rouge_2_score=rouge_2(best_answers['L5'],data_GPT['医生回答'])
# rouge_l_score=rouge_l(best_answers['L5'],data_GPT['医生回答'])
# bleu_4_score=bleu_4(best_answers['L5'],data_GPT['医生回答'])
# 
# print('rouge-1:',rouge_1_score)
# print('rouge-2:',rouge_2_score)
# print('rouge-l:',rouge_l_score)
# print('bleu-4:',bleu_4_score)